# Equations in OR-Tools

This notebook explores additional ways to model equations in OR Tools.

In [1]:
import random
from typing import List

from ortools.linear_solver import pywraplp
import pandas as pd
from orlab.exercise6 import tank_solve

In this simple example, we have 3 liquid tanks, each with a different capacity.  Each of the tanks has an amount of starting liquid in it at the start.  We would like the tanks to be as close to 90% full as we can.  We need to choose exactly one of the tanks to add some liquid to, and all of the newly added liquid must go into exactly one of the tanks.  Also each of the tanks has a max level for safety that has been established by the mechanical engineers.

We want to model `deviation = abs(target_level - fill_level)`, but we are also trying to keep things linear for our modelling of the problem.  `abs()` is not linear, so we need to do a trick to allow us to model this.  The trick is to introduce an extra decision variable for each tank `deviation` and bind it to the absolute value with two inequalities which *are* linear:
* *deviation* >= *expr*
* *deviation* >= *-expr*

Now if we minimize `deviation` that will be equivalent to minimizing `|expr|`.

Let's design some data so that when we see the solution we know it did the right thing:

In [2]:
target_fill_ratio = 0.9
tanks_df = pd.DataFrame({
    'capacity': [100, 100, 100],
    'max_level': 95,
    'current_level': [20, 0, 85],
})
tanks_df['target_level'] = target_fill_ratio * tanks_df.capacity
tanks_df['available_volume'] = tanks_df.target_level - tanks_df.current_level
tanks_df['notes'] = [
        'Add 67: comprised of 11 + 56',
        'Add 40',
        'Add 2',
    ]

tanks_df

,capacity,max_level,current_level,target_level,available_volume,notes
0,100,95,20,90.0,70.0,Add 67: comprised of 11 + 56
1,100,95,0,90.0,90.0,Add 40
2,100,95,85,90.0,5.0,Add 2


In [3]:
tank_capacities = [100, 100, 100]
current_fill_levels = [20, 0, 85]
max_fill_levels = [95, 95, 95]
target_fill_ratio = 0.9  # We want the tanks 90% full to the extent we can.
all_tanks = range(len(tank_capacities))

all_demands = [11, 56, 40, 2] # The proposed demands design to fit into particular tanks.
expected_tanks = [0, 0, 1, 2] # The tank numbers we expect these to go to, for checking

In [4]:
model = pywraplp.Solver.CreateSolver("CBC")

In [5]:
# Decision variables
x_vars = {} # x_vars[demand, tank]: For each demand, which tank do we put it in
for demand in all_demands:
    for tank in all_tanks:
        x_vars[demand, tank] = model.BoolVar(f'fill_{demand}_{tank}')


In [6]:
# Constraint: Only one tank is chosen per demand
for demand in all_demands:
    tanks_used = [x_vars[demand, tank] for tank in all_tanks]
    model.Add(sum(tanks_used) == 1)

In [7]:
# Objective: get tanks close to target level
tank_deviations = [...] # We're not doing this, for the reasons discussed below.

---

## Issues with the above formulation

* Trying to minimize the deviation using absolute values isn't going to work. No matter where we choose to put new demands, the sum of the deviations is always going to be the same.  If we used squared deviation this would be better, but that's non-linear.  There's nothing stopping it from putting small demands on pristine empty tanks, which we'd be better off saving for later.  Adding several empty tanks to tempt it would reveal this.
* The "target" level thing is probably a heuristic, rather than an actual goal.  The actual goal is to load our tanks as efficiently as we can subject to the constraints.  The target level thing will make us perfectly optimize for something suboptimal, which subverts the whole reason we're using a solver for in the first place.

What we're actually trying to do here is make choices to protect ourselves from the uncertainty of the future demand.  If we have perfect knowledge of our future demand then this becomes a simple exercise of placing all that.  But the reality is that we don't have perfect knowledge of the future demand. We have rough ideas and guesses about it, based on what we know about the past and what we know about the future, but we don't have perfect knowledge of future demand.  So we need to make choices in a way that we're in the best position we can be for satisfying the future demand.


## Approach ideas

**Monte Carlo**.  We could simulate future demands and place those as well, after we place our actual known demands.  We could do this many times, and do the optimization based on how many additional demands we were able to place.  This would force the choices for the known demands to be made in a way that puts us in the best position to handle future demands.

**Localized Utilization Efficiency.** But let's think of what we're really trying to do here.  Whenever we choose a tank, we need to choose the tank where the remaining capacity best matches what we're trying to place there.  We just want to leave every tank either untouched, or if we do touch it, we want to fill it as much as possible.  So what if we were to 
directly model that as an optimization parameter? **If we touch a tank, optimize how close to full it was in the end.**

So we're going to try an **objective function that minimizes the sum of the remaining capacity for the tanks we touched.**


## Local Utilization Efficiency

Let's do a new formulation of this that gives us a better playground for seeing how this works.  Changes we're making compared to what we were doing before:
1. Drop the `target_level`.  That was a heuristic rather than being a real thing we're trying to optimize for.
2. Add an additional tank to give the solver the opportunity to use a pristine tank when it should not.


In [ ]:
tanks_df = pd.DataFrame({
    'capacity': [100, 100, 100, 100],
    'max_level': 95,
    'current_level': [20, 0, 80, 0],
})
tanks_df
#all_demands = tanks_df.expected_demands.explode().tolist()
#all_demands


,capacity,max_level,current_level
0,100,95,20
1,100,95,0
2,100,95,80
3,100,95,0


In [9]:
def add_expectations(raw_df: pd.DataFrame, expected_demands: List[List[int]]) -> pd.DataFrame:
    """Add information about what we expect to be placed and analysis."""
    df = raw_df.copy()
    df['available_volume'] = df.max_level - df.current_level
    df['expected_demands'] = expected_demands
    df['expected_add'] = df['expected_demands'].apply(lambda x: sum(x))
    df['new_level'] = df.current_level + df.expected_add
    df['new_utilization'] = df.new_level / df.max_level
    df['did_add'] = (df['expected_add'] > 0).astype(int) # True if we added something to this tank.
    # In places where we added something to the tank then add the unused or stranded capacity.
    df['unused_capacity'] = (df.max_level - df.new_level) * df.did_add
    return df

In [10]:
expected_demands = [[18,56], [40], [2], []]
add_expectations(tanks_df, expected_demands)

,capacity,max_level,current_level,available_volume,expected_demands,expected_add,new_level,new_utilization,did_add,unused_capacity
0,100,95,20,75,"[18, 56]",74,94,0.989474,1,1
1,100,95,0,95,[40],40,40,0.421053,1,55
2,100,95,80,15,[2],2,82,0.863158,1,13
3,100,95,0,95,[],0,0,0.000000,0,0


Now notice if we had instead chosen to put the small `2` demand onto the empty row, the `unused_capacity` numbers look much different:

In [11]:
expected_demands = [[18,56], [40], [], [2]]
add_expectations(tanks_df, expected_demands)

,capacity,max_level,current_level,available_volume,expected_demands,expected_add,new_level,new_utilization,did_add,unused_capacity
0,100,95,20,75,"[18, 56]",74,94,0.989474,1,1
1,100,95,0,95,[40],40,40,0.421053,1,55
2,100,95,80,15,[],0,80,0.842105,0,0
3,100,95,0,95,[2],2,2,0.021053,1,93


## Solving using Utilization Efficiency

The strategy we'll go for here is to optimize toward the minimum sum of `unused_capacity`.  The unused capacity isn't the full unused capacity, but only includes the unused capacities for things that we touched.

We could optimize towards making the sum of those `new_utilization` numbers as low as possible, and that would sort of do what we want.  But instead our strategy will effectively ignore the `new_utilization` ratios. Instead we'll be focusing on the absolute numbers of the unused or stranded capacity (only when we deployed a demand there).  This is because we care more about the total magnitude of the stranded capacity than about the precise ways in which that fragmentation is happening.

Let's separate our problem into 3 components so that we can easily try different scenarios:
1. State (how many tanks we have, and how much is in each)
2. Demand
3. Results

Then we'll make a function that takes `(state, demand)` and returns a result.

### Modelling the State

`tanks_df` already models the state pretty well, so we'll just use that like it is.

In [12]:
tanks_df

,capacity,max_level,current_level
0,100,95,20
1,100,95,0
2,100,95,80
3,100,95,0


### Modelling demand

We'll model demands as a list of numbers representing the volumes that need to be placed somewhere in a tank.

In [13]:
demands = [18, 56, 40, 2]
random.shuffle(demands) # Shuffle because placing them top down is what we expect. Shake it up.
demands

[40, 56, 2, 18]

### Solver Code

Our solver will be a function that takes in the world state as a dataframe and the demands as a list.

For modelling the results, we'll return a dataframe so this can be easily visualized.

In [14]:
ok, result_df = tank_solve(tanks_df, demands)
ok

tank 0 placed [56] stranded_vars[tank]=19.0
tank 1 placed [18] stranded_vars[tank]=77.0
tank 2 placed [2] stranded_vars[tank]=13.0
tank 3 placed [40] stranded_vars[tank]=55.0


True

In [15]:
result_df

,capacity,max_level,current_level,demands,placed,new_level,stranded_var,stranded
0,100,95,20,[56],56,76,19.0,19
1,100,95,0,[18],18,18,77.0,77
2,100,95,80,[2],2,82,13.0,13
3,100,95,0,[40],40,40,55.0,55


In [16]:
random.shuffle(demands)
ok, result_df = tank_solve(tanks_df, demands)
result_df

tank 0 placed [40] stranded_vars[tank]=35.0
tank 1 placed [18] stranded_vars[tank]=77.0
tank 2 placed [2] stranded_vars[tank]=13.0
tank 3 placed [56] stranded_vars[tank]=39.0


,capacity,max_level,current_level,demands,placed,new_level,stranded_var,stranded
0,100,95,20,[40],40,60,35.0,35
1,100,95,0,[18],18,18,77.0,77
2,100,95,80,[2],2,82,13.0,13
3,100,95,0,[56],56,56,39.0,39


In [17]:
demands

[18, 56, 40, 2]

For later consideration:

- [ ] Can we optimize `remaining[tank] - x_vars[tank, demand] * qty[demand]` to minimize?

---

## Try solution B

In [18]:
from orlab.exercise6b import tank_solve_b

In [20]:
ok, result_df = tank_solve_b(tanks_df, demands)
ok

True

In [21]:
result_df

,capacity,max_level,current_level,demands,placed,new_level,stranded
0,100,95,20,"[56.0, 2.0]",58.0,78.0,17.0
1,100,95,0,"[18.0, 40.0]",58.0,58.0,37.0
2,100,95,80,[],0.0,80.0,0.0
3,100,95,0,[],0.0,0.0,0.0
